In [3]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
import tempfile
TEMP_FOLDER = tempfile.gettempdir()
print('Folder "{}" will be used to save temporary dictionary and corpus.'.format(TEMP_FOLDER))

#%matplotlib inline
#import matplotlib
#import matplotlib.pyplot as plt
#from IPython.display import set_matplotlib_formats
#set_matplotlib_formats('retina')

from sklearn import preprocessing

import warnings
warnings.filterwarnings(action='ignore', category=UserWarning, module='gensim')
import gensim

Folder "C:\Users\cchen07\AppData\Local\Temp" will be used to save temporary dictionary and corpus.


### Read the pandas dataframe we got from data preparation

In [5]:
root_folder = Path('../Data/Pickle/')
file_name = 'TRACE2014_jinming'
file_path = root_folder / file_name

data = pd.read_pickle(file_path)

#### Group data by ['document','BOND_SYM_ID_transformed'] so that represent a document and token inside it

In [7]:
data_gb = data.groupby(by=['document','BOND_SYM_ID'])

In [8]:
matrix_1 = data_gb['BOND_SYM_ID'].size().unstack(fill_value=0)
matrix_1 = matrix_1.sort_index(axis=1)

matrix_1_shape = matrix_1.shape
print('We have {} documents and {} tokens'.format(matrix_1_shape[0],matrix_1_shape[1]))

We have 98304 documents and 12078 tokens


### Matrix 1 => Count of  BOND_SYM_ID in a document

#### Use SKlearn Label Encoder to transform tokens into integer

In [10]:
le = preprocessing.LabelEncoder()
le.fit(matrix_1.columns)
transform = le.transform(matrix_1.columns)
inverse_transform = le.inverse_transform(transform)

C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Create id2word to map label encoding to its words then create corpus with gensim Dense2Corpus which takes numpy

In [13]:
id2word = dict(zip(transform, inverse_transform))

matrix1_corpus = gensim.matutils.Dense2Corpus(matrix_1.values,documents_columns=False)
# Run LDA in Gensim
lda = gensim.models.ldamulticore.LdaMulticore(corpus= matrix1_corpus,id2word=id2word,workers=3, num_topics=500, chunksize=10000, passes=1)

2018-11-16 13:39:59,923 : INFO : using symmetric alpha at 0.002
2018-11-16 13:40:00,021 : INFO : using symmetric eta at 0.002
2018-11-16 13:40:00,202 : INFO : using serial LDA version on this node
2018-11-16 13:40:08,863 : INFO : running online LDA training, 500 topics, 1 passes over the supplied corpus of 98304 documents, updating every 30000 documents, evaluating every ~98304 documents, iterating 50x with a convergence threshold of 0.001000
2018-11-16 13:40:08,863 : WARNING : too few updates, training might not converge; consider increasing the number of passes or iterations to improve accuracy
2018-11-16 13:40:09,845 : INFO : training LDA model using 3 processes
2018-11-16 13:40:16,624 : INFO : PROGRESS: pass 0, dispatched chunk #0 = documents up to #10000/98304, outstanding queue size 1
2018-11-16 13:40:28,952 : INFO : PROGRESS: pass 0, dispatched chunk #1 = documents up to #20000/98304, outstanding queue size 2
2018-11-16 13:40:40,149 : INFO : PROGRESS: pass 0, dispatched chunk #2

In [14]:
for idx, topic in lda.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

2018-11-16 13:46:38,127 : INFO : topic #0 (0.002): 0.010*"EXC.DE" + 0.004*"FST.GO" + 0.004*"FCX3981382" + 0.004*"AMT3951226" + 0.004*"CAH.AA" + 0.003*"PBR4006643" + 0.003*"AAPL4001809" + 0.003*"NEM3827984" + 0.003*"CLF.AD" + 0.003*"VZ4050437"
2018-11-16 13:46:38,128 : INFO : topic #1 (0.002): 0.047*"RAAM.AD" + 0.006*"ARCC4071336" + 0.006*"GNW.JO" + 0.006*"AMGN.HA" + 0.004*"RIG.GP" + 0.004*"SWY.HG" + 0.004*"RRD.AA" + 0.004*"GNW.GA" + 0.003*"SVU4076834" + 0.003*"BTU3905482"
2018-11-16 13:46:38,129 : INFO : topic #2 (0.002): 0.013*"BK3704406" + 0.010*"UNP.LH" + 0.007*"NTXX4067232" + 0.006*"ENLA3704337" + 0.006*"ESV3684248" + 0.005*"CLF.AD" + 0.005*"NE.AE" + 0.003*"SWY.HG" + 0.003*"PPL.IR" + 0.003*"LINE3690892"
2018-11-16 13:46:38,130 : INFO : topic #3 (0.002): 0.012*"PHM.GL" + 0.009*"CLF.AC" + 0.009*"F.GY" + 0.008*"APU.AB" + 0.006*"FTR3669882" + 0.006*"GE4105158" + 0.005*"OAS4155695" + 0.005*"T4013484" + 0.004*"SWY.GN" + 0.004*"DELL.GH"
2018-11-16 13:46:38,131 : INFO : topic #4 (0.002): 0

2018-11-16 13:46:38,159 : INFO : topic #34 (0.002): 0.016*"FCX4061104" + 0.016*"WCRX.AC" + 0.006*"JCP.GP" + 0.006*"AMD4155878" + 0.004*"BTU.GI" + 0.004*"PEP3674727" + 0.003*"T3818484" + 0.003*"HBC.AGZ" + 0.003*"PBR4006643" + 0.003*"CLF.AC"
2018-11-16 13:46:38,160 : INFO : topic #35 (0.002): 0.006*"SWY.GN" + 0.005*"SWY.GH" + 0.004*"ETP.GM" + 0.004*"AU.GD" + 0.004*"CIB3898329" + 0.003*"CLF.AC" + 0.003*"NEM3827984" + 0.003*"DUK3668316" + 0.003*"AU3882696" + 0.003*"PBR3688575"
2018-11-16 13:46:38,161 : INFO : topic #36 (0.002): 0.010*"KEG.GI" + 0.007*"PBR3688575" + 0.006*"CTB.GD" + 0.005*"AAPL4001809" + 0.005*"PBR4006643" + 0.005*"PPL.IR" + 0.004*"CLF.AC" + 0.004*"CLF.AD" + 0.004*"CTL3705105" + 0.003*"VZ4050437"
2018-11-16 13:46:38,163 : INFO : topic #37 (0.002): 0.007*"DVA.AB" + 0.005*"TGT4136096" + 0.005*"FSIC4142507" + 0.005*"AAPL4001809" + 0.005*"GSK3981912" + 0.005*"HL4080028" + 0.004*"PBR3688575" + 0.004*"MYL4072750" + 0.003*"PBR4006643" + 0.003*"SWY.GN"
2018-11-16 13:46:38,164 : INF

2018-11-16 13:46:38,198 : INFO : topic #68 (0.002): 0.036*"RCII.GH" + 0.024*"IEP4121535" + 0.006*"SWY.GN" + 0.006*"CSC3901509" + 0.005*"NES3879264" + 0.005*"CLF.AD" + 0.005*"CDFA.GC" + 0.004*"CCCX3917899" + 0.004*"SWY.GH" + 0.003*"CLF.AA"
2018-11-16 13:46:38,199 : INFO : topic #69 (0.002): 0.033*"GMBX3926765" + 0.018*"DB4009155" + 0.018*"HRZL3929403" + 0.013*"CIB3898329" + 0.006*"AU.GD" + 0.006*"LTD.GJ" + 0.005*"RBS4081543" + 0.005*"PBR4006643" + 0.004*"SFTBF3928946" + 0.004*"CLF.AB"
2018-11-16 13:46:38,200 : INFO : topic #70 (0.002): 0.012*"KO.GT" + 0.006*"ADT4104761" + 0.005*"FCX3699393" + 0.004*"SNH4119686" + 0.003*"SFTBF3928946" + 0.003*"SWY.GN" + 0.003*"CLF.AC" + 0.003*"CLF.AD" + 0.003*"BTU.GI" + 0.002*"PPL.IR"
2018-11-16 13:46:38,202 : INFO : topic #71 (0.002): 0.010*"BX3687424" + 0.004*"SVU3706786" + 0.004*"BHP3822195" + 0.004*"PBR4006643" + 0.003*"CVX3938386" + 0.003*"C.HTX" + 0.003*"TD4180383" + 0.003*"VZ4050437" + 0.003*"TD3827890" + 0.003*"RIO3890854"
2018-11-16 13:46:38,203

2018-11-16 13:46:38,230 : INFO : topic #102 (0.002): 0.050*"AU4033717" + 0.009*"SWY.GN" + 0.008*"RIG.HE" + 0.006*"SWY.HG" + 0.006*"IEP4122989" + 0.005*"RIG.HG" + 0.004*"SFTBF3928946" + 0.004*"RIG3684663" + 0.004*"AMD.AA" + 0.004*"FCFS4163741"
2018-11-16 13:46:38,231 : INFO : topic #103 (0.002): 0.017*"AVP.GK" + 0.007*"JPM3889542" + 0.007*"CLF.AC" + 0.004*"PBR4006643" + 0.003*"AAPL4001809" + 0.003*"SWY.GN" + 0.003*"PBR4006641" + 0.003*"PBR3688575" + 0.003*"AA.HO" + 0.003*"BRK4059873"
2018-11-16 13:46:38,231 : INFO : topic #104 (0.002): 0.036*"FST.GO" + 0.020*"SWY.GH" + 0.010*"AMD4013916" + 0.010*"PBR3674838" + 0.010*"SWY.GN" + 0.008*"FST4035582" + 0.007*"LXP4105866" + 0.005*"PVA.GD" + 0.004*"CDFA.GC" + 0.004*"RRD3673769"
2018-11-16 13:46:38,232 : INFO : topic #105 (0.002): 0.010*"VIA.HD" + 0.006*"BGC.GJ" + 0.006*"HD.GM" + 0.005*"TWC3672023" + 0.005*"CSCO.GE" + 0.005*"CMCS.GZ" + 0.004*"RDS4038461" + 0.004*"BP.JH" + 0.004*"AZN.GD" + 0.004*"TWC3677006"
2018-11-16 13:46:38,233 : INFO : topi

2018-11-16 13:46:38,254 : INFO : topic #136 (0.002): 0.008*"KKR4105360" + 0.006*"SWY.GN" + 0.005*"KMI3667609" + 0.005*"HCP.JK" + 0.005*"RTSX.AC" + 0.004*"LOW.HC" + 0.004*"DTEGF4161122" + 0.004*"SFTBF3928946" + 0.003*"PBR3688575" + 0.003*"AVP3979750"
2018-11-16 13:46:38,255 : INFO : topic #137 (0.002): 0.026*"JBL.GG" + 0.019*"AN.GG" + 0.015*"CNK3940128" + 0.013*"JPM.QDN" + 0.011*"AXP3962890" + 0.011*"GNW.JP" + 0.007*"GNW.AA" + 0.006*"FIS3881268" + 0.006*"BLKG.AB" + 0.005*"AGN3979700"
2018-11-16 13:46:38,256 : INFO : topic #138 (0.002): 0.019*"PBI4104274" + 0.018*"SFG.GB" + 0.007*"IBM4096360" + 0.007*"PPL.IR" + 0.006*"BRK3695121" + 0.004*"CDFA.GC" + 0.004*"SWY.GN" + 0.004*"WIN3982620" + 0.004*"PBR4006643" + 0.004*"AXP.LA"
2018-11-16 13:46:38,257 : INFO : topic #139 (0.002): 0.051*"AGO4136090" + 0.018*"SWY.GN" + 0.013*"GMRK3956992" + 0.012*"IEP4121535" + 0.009*"EAC4123496" + 0.008*"BTU.GI" + 0.007*"RIG.GX" + 0.006*"DTEGF4060201" + 0.006*"WLT4062096" + 0.005*"MX4029826"
2018-11-16 13:46:38

...

2018-11-16 13:46:38,450 : INFO : topic #374 (0.002): 0.032*"SNV.AA" + 0.010*"OIS.GC" + 0.008*"TWX.BD" + 0.005*"HRS.GI" + 0.005*"CLW.GE" + 0.004*"DRL.GH" + 0.004*"GM3827076" + 0.004*"GCI3668096" + 0.004*"SFTBF3928946" + 0.003*"CLF.AD"
2018-11-16 13:46:38,451 : INFO : topic #375 (0.002): 0.007*"CLD.AA" + 0.005*"AGO4136090" + 0.004*"FE.GS" + 0.004*"PBR3688575" + 0.004*"LBTY3954936" + 0.003*"SWY.GN" + 0.003*"RDEN.GD" + 0.003*"VZ4050437" + 0.003*"T3941564" + 0.003*"PPL.IR"
2018-11-16 13:46:38,451 : INFO : topic #376 (0.002): 0.075*"NSM4013441" + 0.029*"VNR3840075" + 0.010*"NWL4184959" + 0.007*"BRK4133973" + 0.006*"LINE3690892" + 0.005*"CLF.AC" + 0.005*"SVU4183445" + 0.005*"NOR4097709" + 0.004*"RIG.HE" + 0.004*"CIB3898329"
2018-11-16 13:46:38,452 : INFO : topic #377 (0.002): 0.012*"FTR3669882" + 0.010*"DG3992947" + 0.008*"PBR4006643" + 0.006*"CLF.AC" + 0.005*"CLF3941513" + 0.004*"PBR3688575" + 0.004*"SFTBF3928946" + 0.004*"SEM4048510" + 0.004*"CLF.AD" + 0.003*"NEM3827984"
2018-11-16 13:46:38

2018-11-16 13:46:38,478 : INFO : topic #408 (0.002): 0.008*"LINE3690892" + 0.005*"T4013485" + 0.004*"AAPL4001809" + 0.004*"PSD.IQ" + 0.004*"SFTBF3703037" + 0.004*"BRK3890845" + 0.003*"PBR4006643" + 0.003*"DISH3996225" + 0.003*"AMD.AA" + 0.003*"SBUX.GA"
2018-11-16 13:46:38,479 : INFO : topic #409 (0.002): 0.037*"RCII.GH" + 0.015*"LH.GH" + 0.009*"CLF.AC" + 0.006*"SWY.GH" + 0.006*"SWY.GN" + 0.005*"WLT4062096" + 0.004*"KGC3871484" + 0.004*"CLF.AB" + 0.004*"JPM3999853" + 0.004*"CTL3705105"
2018-11-16 13:46:38,479 : INFO : topic #410 (0.002): 0.007*"JCP.GI" + 0.005*"EGN.IA" + 0.005*"DPS.AI" + 0.004*"PBR3688575" + 0.004*"RDEN.GD" + 0.003*"FNF3891945" + 0.003*"CLF.AC" + 0.003*"DGX.GK" + 0.003*"PRE.GB" + 0.003*"MKL.GJ"
2018-11-16 13:46:38,480 : INFO : topic #411 (0.002): 0.074*"BRK4133973" + 0.026*"EAC4123496" + 0.021*"FTR3887874" + 0.007*"BTU3905482" + 0.006*"LEVI.GR" + 0.005*"WLT4062096" + 0.005*"CSCO.GF" + 0.005*"NIHD.GJ" + 0.004*"CCK4062852" + 0.004*"CCCX3917899"
2018-11-16 13:46:38,481 : I

2018-11-16 13:46:38,507 : INFO : topic #442 (0.002): 0.019*"SNPO3982660" + 0.006*"VOD3968277" + 0.006*"CGG3698619" + 0.006*"VNR3840075" + 0.005*"CDFA.GC" + 0.004*"KMB.AB" + 0.004*"ARCP4095545" + 0.004*"SFTBF3928946" + 0.003*"TWI4136990" + 0.003*"TEVA.GU"
2018-11-16 13:46:38,508 : INFO : topic #443 (0.002): 0.005*"PBR3706410" + 0.004*"PBR4006641" + 0.004*"EAC4123496" + 0.003*"PBR3688575" + 0.003*"AU4033717" + 0.003*"FST.GO" + 0.003*"RS3993406" + 0.003*"MCP3983624" + 0.003*"PBR4006643" + 0.002*"AXE3849290"
2018-11-16 13:46:38,509 : INFO : topic #444 (0.002): 0.015*"HCA.GF" + 0.012*"STT.GT" + 0.004*"PBR4006643" + 0.004*"CLF.AC" + 0.004*"L4002796" + 0.004*"CCCX3917899" + 0.004*"SD.AE" + 0.003*"TOY3916828" + 0.003*"SWY.GN" + 0.003*"SD.AH"
2018-11-16 13:46:38,510 : INFO : topic #445 (0.002): 0.023*"CTB.GD" + 0.018*"ADT3991008" + 0.009*"SGY3921538" + 0.009*"CLMT4081172" + 0.006*"PPL.IR" + 0.005*"KEG.GI" + 0.005*"PBR4106369" + 0.005*"CLF.AB" + 0.004*"AGN.KV" + 0.003*"SFTBF3928946"
2018-11-16 1

2018-11-16 13:46:38,538 : INFO : topic #475 (0.002): 0.023*"KWK.GC" + 0.020*"AGO.GB" + 0.010*"BTU3905481" + 0.010*"CLF.AD" + 0.009*"HOS3982287" + 0.009*"WLT4062096" + 0.008*"SETL4035995" + 0.006*"AU4033717" + 0.006*"SWY.GN" + 0.006*"SWY.HG"
2018-11-16 13:46:38,538 : INFO : topic #476 (0.002): 0.042*"BLL4005574" + 0.012*"WMT.IG" + 0.008*"PBR4006643" + 0.008*"NEM3827984" + 0.006*"JCP.GI" + 0.005*"PBR4106372" + 0.005*"AU3882696" + 0.004*"PBR3688575" + 0.004*"SFTBF3703037" + 0.004*"PPL.IR"
2018-11-16 13:46:38,539 : INFO : topic #477 (0.002): 0.006*"PFE.GO" + 0.006*"TLM.GE" + 0.006*"COF.HL" + 0.005*"BGC.GJ" + 0.005*"NBR.GR" + 0.005*"OXY.HM" + 0.005*"MDLZ3677598" + 0.005*"BP.JH" + 0.005*"STO4005129" + 0.004*"GP.GV"
2018-11-16 13:46:38,540 : INFO : topic #478 (0.002): 0.017*"PPL.IC" + 0.016*"SAI3704710" + 0.005*"CLF.AC" + 0.005*"SWY.GN" + 0.004*"MUR3856177" + 0.003*"CACC4090304" + 0.003*"PPL.IR" + 0.003*"WU.GF" + 0.003*"AU4033717" + 0.003*"GMBX3926765"
2018-11-16 13:46:38,541 : INFO : topic #

Topic: 0 
Words: 0.010*"EXC.DE" + 0.004*"FST.GO" + 0.004*"FCX3981382" + 0.004*"AMT3951226" + 0.004*"CAH.AA" + 0.003*"PBR4006643" + 0.003*"AAPL4001809" + 0.003*"NEM3827984" + 0.003*"CLF.AD" + 0.003*"VZ4050437"
Topic: 1 
Words: 0.047*"RAAM.AD" + 0.006*"ARCC4071336" + 0.006*"GNW.JO" + 0.006*"AMGN.HA" + 0.004*"RIG.GP" + 0.004*"SWY.HG" + 0.004*"RRD.AA" + 0.004*"GNW.GA" + 0.003*"SVU4076834" + 0.003*"BTU3905482"
Topic: 2 
Words: 0.013*"BK3704406" + 0.010*"UNP.LH" + 0.007*"NTXX4067232" + 0.006*"ENLA3704337" + 0.006*"ESV3684248" + 0.005*"CLF.AD" + 0.005*"NE.AE" + 0.003*"SWY.HG" + 0.003*"PPL.IR" + 0.003*"LINE3690892"
Topic: 3 
Words: 0.012*"PHM.GL" + 0.009*"CLF.AC" + 0.009*"F.GY" + 0.008*"APU.AB" + 0.006*"FTR3669882" + 0.006*"GE4105158" + 0.005*"OAS4155695" + 0.005*"T4013484" + 0.004*"SWY.GN" + 0.004*"DELL.GH"
Topic: 4 
Words: 0.057*"CF.AA" + 0.017*"GADN3706985" + 0.010*"FBR4124761" + 0.009*"BEAV.AB" + 0.007*"DLNG4162479" + 0.006*"BA.GCH" + 0.006*"NEM.GS" + 0.005*"RIG.HE" + 0.004*"CDFA.GC" + 0.0

In [21]:
from gensim.test.utils import datapath
temp_file = datapath("matrix_1_lda")
lda.save(temp_file)

2018-11-16 14:11:30,500 : INFO : saving LdaState object under C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\test\test_data\../matrix_1_lda/matrix_1_lda.state, separately None


FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\cchen07\\AppData\\Local\\Continuum\\anaconda3\\lib\\site-packages\\gensim\\test\\test_data\\../matrix_1_lda/matrix_1_lda.state'

In [19]:
matrix_1 = gensim.models.ldamodel.LdaModel.load(temp_file)

2018-11-16 14:10:47,203 : INFO : loading LdaModel object from C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\test\test_data\matrix_1_lda
2018-11-16 14:10:47,281 : INFO : loading expElogbeta from C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\test\test_data\matrix_1_lda.expElogbeta.npy with mmap=None
2018-11-16 14:10:47,416 : INFO : setting ignored attribute id2word to None
2018-11-16 14:10:47,416 : INFO : setting ignored attribute dispatcher to None
2018-11-16 14:10:47,416 : INFO : setting ignored attribute state to None
2018-11-16 14:10:47,416 : INFO : loaded C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\test\test_data\matrix_1_lda
2018-11-16 14:10:47,416 : INFO : loading LdaState object from C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site-packages\gensim\test\test_data\matrix_1_lda.state
2018-11-16 14:10:47,547 : INFO : loaded C:\Users\cchen07\AppData\Local\Continuum\anaconda3\lib\site